In [25]:
import numpy as np                        # numerical computations
from matplotlib import pyplot as plt      # visualisation
import xarray as xr                       # for netcdf
import pandas as pd                       
import os

In [33]:
first_file = xr.open_dataset("../data/Archivos_reanalisisERA5_realesINUMET/Reanalisis/Datos_t2m_horario_2000a2010_uy.nc")
first_file.head()

<xarray.Dataset>
Dimensions:    (longitude: 5, latitude: 5, time: 5)
Coordinates:
  * longitude  (longitude) float32 -58.0 -57.75 -57.5 -57.25 -57.0
  * latitude   (latitude) float32 -30.0 -30.25 -30.5 -30.75 -31.0
  * time       (time) datetime64[ns] 2000-01-01 ... 2000-01-01T04:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-12-13 16:08:08 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [35]:
def prepare_data(file):
    df = pd.DataFrame()

    df["time"] = pd.to_datetime(file.time)
    df["t2m"] = file.t2m.mean(dim=['latitude', 'longitude'])
    df.set_index("time")

    # type of the date column -> datetime64[ns] !!
    # separating the time column
    df["date"] = df["time"].dt.date
    df["day"] = df["time"].dt.day
    df["month"] = df["time"].dt.month

    # changing the format of the date column by removing the hours

    # in .groupby only 2 columns date and t2m are taken from the groupings
    # in .agg the dict is used to apply those functions only to t2m column
    df_by_day = df.groupby("date")[["date", "t2m"]].agg({"t2m": ["mean", "min", "max"]}).rename(columns={"t2m":"t2m_daily"})   # temp precipitation becomes daily and not hourly mesured

    df_by_day.columns = ['_'.join(col).strip() for col in df_by_day.columns.values]
    return df_by_day

prepare_data(first_file)

,t2m_daily_mean,t2m_daily_min,t2m_daily_max
date,,,
2000-01-01,295.357391,289.210754,301.250488
2000-01-02,295.893982,290.180756,301.432617
2000-01-03,296.112152,289.388184,302.286987
2000-01-04,296.718018,290.670929,302.490143
2000-01-05,298.134247,292.307312,304.380524
...,...,...,...
2010-12-27,298.983368,292.048462,305.240265
2010-12-28,297.549103,290.811432,303.660004
2010-12-29,297.873322,293.201782,302.823700


In [14]:
second_file = xr.open_dataset("../data/Archivos_reanalisisERA5_realesINUMET/Reanalisis/Datos_t2m_horario_2011a2021_uy.nc")
second_file.head()

<xarray.Dataset>
Dimensions:    (longitude: 5, latitude: 5, time: 5)
Coordinates:
  * longitude  (longitude) float32 -58.0 -57.75 -57.5 -57.25 -57.0
  * latitude   (latitude) float32 -30.0 -30.25 -30.5 -30.75 -31.0
  * time       (time) datetime64[ns] 2011-01-01 ... 2011-01-01T04:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-12-12 19:36:09 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [36]:
prepare_data(second_file)

,t2m_daily_mean,t2m_daily_min,t2m_daily_max
date,,,
2011-01-01,297.369751,293.537048,301.701630
2011-01-02,297.225861,293.782959,301.546265
2011-01-03,296.497528,293.960083,299.651703
2011-01-04,297.186340,294.171387,301.291840
2011-01-05,298.048798,293.897675,302.858765
...,...,...,...
2021-12-27,297.730194,290.790375,304.073547
2021-12-28,298.024994,289.748260,305.743622
2021-12-29,299.656403,292.500610,306.912903
